# Advent of Code 2017: Day 3

## Problem statement

> You come across an experimental new kind of memory stored on an <font color='green'>infinite two-dimensional grid</font>.

> Each square on the grid is allocated in a <font color='green'>spiral pattern starting at a location marked 1 and then counting up while spiraling outward</font>. For example, the first few squares are allocated like this:

> |      |      |      |      |      |      |      |
  | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
  |      |  17  |  16  |  15  |  14  |  13  |      |
  |      |  18  |   5  |   4  |   3  |  12  |      |
  |      |  19  |   6  | **1**|   2  |  11  |      |
  |      |  20  |   7  |   8  |   9  |  10  |      |
  |      |  21  |  22  |  23  |  24  | -->  |      |
  |      |      |      |      |      |      |      |      |

> While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs <font color='blue'>that can only move up, down, left, or right</font>. They always take the shortest path: the <font color='red'>Manhattan Distance</font> between the <font color='red'>location of the data</font> and square 1.

> **How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port**?

## Breaking down the problem
- **Task**: Find the shortest path between a given square and square 1
- <font color='green'>Data</font>: An infinite 2D grid of values that increment, spiraling outward from the centre
- <font color='blue'>Constraints</font>: Shortest path will be to move all the way along one axis, and then the other - the length of this is the Manhattan distance
- <font color='red'>Find location of the data</font>: Given the *index* of the data, find the 2D coordinate
- <font color='red'>Manhattan distance</font>: Compute the Manhattan distance from this square to square 1

## Implementation
### Grid movement
These utility functions allow us to move across a grid using only relative operations rather than specifying coordinates exactly. Likewise we can check what is in the square to the left _relative to the direction of movement_.

In [261]:
Location = Direction = complex

def turn_left(forward):
    return Direction(forward * 1j)

def adjacent(location, forward):
    return location + (forward * 1j)

In [262]:
def manhattan_distance(direction):
    return abs(direction.real) + abs(direction.imag)

### Traverse the spiral
To traverse a spiral in order, we start at the centre and iteratively move in a 'forward' direction. This direction is updated such that the adjacent square to the 'left' will always have already been visited.

In [263]:
def spiral(start=Location(0, 0), forward=Direction(0, -1)):
    visited = set([start])
    location = start
    
    yield location

    while True:
        if adjacent(location, forward) not in visited:
            forward = turn_left(forward)
        location += forward

        visited.add(location)
        yield location

### Locate square from index
With movement along a spiral already defined, the location can be found by simply moving the required number of steps. The location back to the port is then the Manhattan distance of this location from the origin.

In [264]:
def distance_from_index(index):
    move = spiral(start=Location(0, 0))
    
    for i in range(index):
        location = next(move)

    return manhattan_distance(location)

## Check against test cases

In [265]:
assert distance_from_index(1) == 0
assert distance_from_index(12) == 3
assert distance_from_index(23) == 2
assert distance_from_index(1024) == 31

## Solve problem

In [266]:
print(distance_from_index(312051))

430.0


For **part two** grid values are still allocated in the same order, however the value in a square is not the index but instead the sum of all adjacent squares *including diagonals* at the time of allocation. For instance, when allocating the second square, it's only allocated neighbour is 1 and so its value is also 1. The next square is adjacent to both of these, and so has value 2, and this continues as the spiral grows infinitely. The first 23 values are:

|      |      |      |      |      |      |      |
| ---: | ---: | ---: | ---: | ---: | ---: | ---: |
|      | 147  | 142  | 133  | 122  |  59  |      |
|      | 304  |   5  |   4  |   2  |  57  |      |
|      | 330  |  10  | **1**|   1  |  54  |      |
|      | 351  |  11  |  23  |  25  |  26  |      |
|      | 362  | 747  | 806  | -->  | ...  |      |
|      |      |      |      |      |      |      |      |

This can be adapted by traversing the spiral in the same manner. Unlike in part one, it is necessary in this case to actually keep a record of the allocated values so that new grid squares can be allocated the correct sum.

In [267]:
def neighbours(location):
    for dy in [-1, 0, 1]:
        for dx in [-1, 0, 1]:
            yield location + Direction(dx, dy)

In [268]:
def fill_squares(max_value):
    grid = {Location(0, 0): 1}

    for location in spiral():
        grid[location] = sum(grid[neighbour]
                             for neighbour in neighbours(location)
                             if neighbour in grid)

        if grid[location] > max_value:
            return grid[location]

In [269]:
print(fill_squares(max_value=312051))

312453
